# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will also be using the watchlist you just created to figure out which streaming services contain the shows that you want to watch next so you can decide which one is worth the money to you.

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. Remember that during the prep work, we learned that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell.

## Database Setup

To start with, import the necessary libraries and create a dataframe from the provided CSV. Print the info out for the dataframe. After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces in the names or unusual characters such as `"+"`. Print out the info for the dataframe again to ensure that your changes were effective.

In [3]:
import pandas as pd
import sqlite3 as sl
data= pd.read_csv("tv_shows.csv")
data=data.drop('Unnamed: 0', 1)
data=data.rename(columns={'Rotten Tomatoes':'RottenT', 'Prime Video': 'Prime', 'Disney+':"Disney"})
data.head()

C:\Users\timle\AppData\Local\Temp/ipykernel_24948/3393260417.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data=data.drop('Unnamed: 0', 1)


,ID,Title,Year,Age,IMDb,RottenT,Netflix,Hulu,Prime,Disney,Type
0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


With your dataframe at the ready, create a new database called `tv`. Add a new table to your database called `shows` using the data in the dataframe. 

In [4]:
con=sl.connect('tv.db')

In [5]:
data.to_sql('show', con)

ValueError: Table 'show' already exists.

With your new table and database set up, print out the top 20 records in the `shows` table.

In [6]:
print(data.head(20))
with con:
        data1 = con.execute("SELECT * FROM show WHERE id<=20")
        for row in data1:
            print(row)

    ID                       Title  Year  Age    IMDb  RottenT  Netflix  Hulu  \
0    1                Breaking Bad  2008  18+  9.4/10  100/100        1     0   
1    2             Stranger Things  2016  16+  8.7/10   96/100        1     0   
2    3             Attack on Titan  2013  18+  9.0/10   95/100        1     1   
3    4            Better Call Saul  2015  18+  8.8/10   94/100        1     0   
4    5                        Dark  2017  16+  8.8/10   93/100        1     0   
5    6  Avatar: The Last Airbender  2005   7+  9.3/10   93/100        1     0   
6    7              Peaky Blinders  2013  18+  8.8/10   93/100        1     0   
7    8            The Walking Dead  2010  18+  8.2/10   93/100        1     0   
8    9                Black Mirror  2011  18+  8.8/10   92/100        1     0   
9   10          The Queen's Gambit  2020  18+  8.6/10   92/100        1     0   
10  11                  Mindhunter  2017  18+  8.6/10   90/100        1     0   
11  12                   Com

Now add a table that includes an id number and the titles on your watchlist.

In [7]:
with con:
    con.execute("""
    CREATE TABLE watchlist(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        titles TEXT
    );
    """)

OperationalError: table watchlist already exists

In [8]:
sql = 'INSERT INTO watchlist (id, titles) values(?,?)'
data2 = [
    (1,'Ozark'),
    (2,'Schitts Creek'),
    (3, 'The Office'),
    (4, 'New Girl'),
    (5, 'This is Us'),
    (6, 'Happy Endings'),
    (7, 'Younger'),
    (8, 'Good Trouble'),
    (9, 'The Bold Type'),
    (10, 'Locke and Key')
]

In [9]:
with con:
    con.executemany(sql, data2)

IntegrityError: UNIQUE constraint failed: watchlist.id

In [10]:
with con:
    data3 = (con.execute('SELECT * FROM watchlist'))
    for row in data3:
        print(row)

(1, 'Ozark')
(2, 'Schitts Creek')
(3, 'The Office')
(4, 'New Girl')
(5, 'This is Us')
(6, 'Happy Endings')
(7, 'Younger')
(8, 'Good Trouble')
(9, 'The Bold Type')
(10, 'Locke and Key')


## Working with the Data

Using either Pandas or SQL, determine how many shows are on each streaming service and what the mean is for each streaming service. 

**Note**: You may notice that the mean represents the percentage of shows in the dataframe that are on each streaming service. Take a moment to think about why that is.

In [11]:
print(data['Netflix'].value_counts())
print(data['Hulu'].value_counts())
print(data['Prime'].value_counts())
print(data['Disney'].value_counts())

print(data['Netflix'].mean())
print(data['Hulu'].mean())
print(data['Prime'].mean())
print(data['Disney'].mean())

0    3397
1    1971
Name: Netflix, dtype: int64
0    3747
1    1621
Name: Hulu, dtype: int64
0    3537
1    1831
Name: Prime, dtype: int64
0    5017
1     351
Name: Disney, dtype: int64
0.3671758569299553
0.3019746646795827
0.34109538002980627
0.06538748137108792


Join your watchlist data with the shows data to determine which streaming services your watchlist shows are on and make a new table in your database using the joined data. Print out the data in your joined table to see what shows on your watchlist on in the original dataset. With the joined data, determine the percentage of your watchlist shows that are on each streaming service and how many of your watchlist shows are on each streaming service.

In [21]:
data4=pd.read_sql("""
     SELECT watchlist.titles, shows.Netflix, shows.Hulu, shows.Prime, shows.Disney 
     FROM watchlist JOIN shows ON shows.Title == watchlist.title;
     """, con)
data4.to_sql('combined', con)


DatabaseError: Execution failed on sql '
     SELECT watchlist.titles, shows.Netflix, shows.Hulu, shows.Prime, shows.Disney 
     FROM watchlist JOIN shows ON shows.Title == watchlist.title;
     ': no such column: shows.Netflix

## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?
2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?
2. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?
3. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?